# Calculate model hyperameters for all pathogens
Uses the [Dask ML](https://ml.dask.org/) implementations of [GridSearchCV](https://ml.dask.org/modules/generated/dask_ml.model_selection.GridSearchCV.html) and [RandomizedSearchCV](https://ml.dask.org/modules/generated/dask_ml.model_selection.RandomizedSearchCV.html) to find hyperparameters for ML models.

The codes executed on the [University of Florida's HiPerGator supercomputer](https://www.rc.ufl.edu/about/hipergator/). The resource allocations were:
* 2 A100 GPUs
* 32 CPU cores
* 64 GB of memory

When possible, GridSearchCV was used. However, this was not feasible for all models. The search algorithm for each model was:
* GridSearchCV
  * Logistic Regression
  * Support Vector Classifier
* RandomizedSearchCV
  * Random Forest Classifer
  * XGBoost Classifier
  * Multilayer Perceptron
  
Note: The `miRNA-analysis` kernel was used to execute the code.

In [1]:
# trying to supress sklearn warning
import os
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import dask_ml.model_selection as dcv
import sklearn

# metrics 
from sklearn.metrics import accuracy_score, f1_score, make_scorer

# models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

# utils
from util import make_mirna_nanostring_df, make_study_df

## helper functions

Use `dask_ml` library for hyperparamerter optimatization (HPO).  
Refs:
- https://docs.rapids.ai/deployment/stable/examples/xgboost-randomforest-gpu-hpo-dask/notebook/#randomsearch
- https://ml.dask.org/modules/generated/dask_ml.model_selection.GridSearchCV.html
- https://ml.dask.org/modules/generated/dask_ml.model_selection.RandomizedSearchCV.html

In [3]:
def score_wrapper(y, y_hat, score_function: f1_score):
    """
    A wrapper function to convert labels to float32,
    and pass it to score_function.

    Params:
    - y: The y labels that need to be converted
    - y_hat: The predictions made by the model
    """
    y = y.astype("float32")  # cuML RandomForest needs the y labels to be float32
    return score_function(y, y_hat, convert_dtype=True)

In [4]:
def exec_HPO(model, gridsearch_params, X, y, scorer=f1_score, n_folds=5, mode="gpu-random", n_iter=25, random_state=42):
    """
    Perform HPO based on the mode specified

    mode: default gpu-Grid. The possible options are:
    1. gpu-grid: Perform GPU based GridSearchCV
    2. gpu-random: Perform GPU based RandomizedSearchCV

    n_iter: specified with Random option for number of parameter settings sampled

    Returns the best estimator and the results of the search
    """
    
    if mode == "gpu-grid":
        print("gpu-grid selected")
        clf = dcv.GridSearchCV(
            model, gridsearch_params, cv=n_folds, scoring=make_scorer(scorer)
        )
    elif mode == "gpu-random":
        print("gpu-random selected")
        clf = dcv.RandomizedSearchCV(
            model, gridsearch_params, cv=n_folds, scoring=make_scorer(scorer), n_iter=n_iter, random_state=random_state
        )

    else:
        print("Unknown Option, please choose one of [gpu-grid, gpu-random]")
        return None, None
    res = clf.fit(X, y)
    print(f"Best clf:\n {res.best_estimator_} \nscore ({scorer.__name__}):\n{res.best_score_}\n---\n")
    return res.best_params_
    # return res.best_estimator_, res


---

## load miRNA data

In [5]:
file_names = [
    '../data/Fusobacterium_nucleatum/8 weeks F. nucleatum infection.csv',
    '../data/Fusobacterium_nucleatum/8 weeks SHAM infection.csv',
    '../data/Fusobacterium_nucleatum/16 weeks F. nucleatum infection.csv',
    '../data/Fusobacterium_nucleatum/16 weeks SHAM infection.csv',
    '../data/Porphyromonas_gingivalis/8 weeks P. gingivalis NanoString Data.csv',
    '../data/Porphyromonas_gingivalis/8 weeks SHAM RAW data.csv',
    '../data/Porphyromonas_gingivalis/16 weeks P. gingivalis  NanoString Data.csv',
    '../data/Porphyromonas_gingivalis/16 weeks SHAM RAW data.csv',
    '../data/Streptococcus_gordonii/8 weeks S.gordonii infection.csv',
    '../data/Streptococcus_gordonii/8 weeks SHAM infection.csv',
    '../data/Streptococcus_gordonii/16 weeks S. gordonii infection.csv',
    '../data/Streptococcus_gordonii/16 weeks SHAM infection.csv',
    '../data/Tannerella_forsythia/8 weeks T. forsythia bacteria infected mice RAW data.csv',
    '../data/Tannerella_forsythia/8 weeks SHAM RAW data.csv',
    '../data/Tannerella_forsythia/16 weeks T. forsythia bacteria infected mice RAW data.csv',
    '../data/Tannerella_forsythia/16 weeks SHAM RAW data.csv',
    '../data/Treponema_denticola/8 weeks T. denticola bacteria infected mice RAW data.csv',
    '../data/Treponema_denticola/8 weeks SHAM RAW data.csv',
    '../data/Treponema_denticola/16 weeks T. denticola bacteria infected mice RAW data.csv',
    '../data/Treponema_denticola/16 weeks SHAM RAW data.csv'
]
cohort_names = [
    'inf_fn_8_weeks', 'sham_fn_8_weeks', 'inf_fn_16_weeks', 'sham_fn_16_weeks',
    'inf_pg_8_weeks', 'sham_pg_8_weeks', 'inf_pg_16_weeks', 'sham_pg_16_weeks',
    'inf_sg_8_weeks', 'sham_sg_8_weeks', 'inf_sg_16_weeks', 'sham_sg_16_weeks',
    'inf_tf_8_weeks', 'sham_tf_8_weeks', 'inf_tf_16_weeks', 'sham_tf_16_weeks',
    'inf_td_8_weeks', 'sham_td_8_weeks', 'inf_td_16_weeks', 'sham_td_16_weeks'
]
list(zip(file_names, cohort_names))

[('../data/Fusobacterium_nucleatum/8 weeks F. nucleatum infection.csv',
  'inf_fn_8_weeks'),
 ('../data/Fusobacterium_nucleatum/8 weeks SHAM infection.csv',
  'sham_fn_8_weeks'),
 ('../data/Fusobacterium_nucleatum/16 weeks F. nucleatum infection.csv',
  'inf_fn_16_weeks'),
 ('../data/Fusobacterium_nucleatum/16 weeks SHAM infection.csv',
  'sham_fn_16_weeks'),
 ('../data/Porphyromonas_gingivalis/8 weeks P. gingivalis NanoString Data.csv',
  'inf_pg_8_weeks'),
 ('../data/Porphyromonas_gingivalis/8 weeks SHAM RAW data.csv',
  'sham_pg_8_weeks'),
 ('../data/Porphyromonas_gingivalis/16 weeks P. gingivalis  NanoString Data.csv',
  'inf_pg_16_weeks'),
 ('../data/Porphyromonas_gingivalis/16 weeks SHAM RAW data.csv',
  'sham_pg_16_weeks'),
 ('../data/Streptococcus_gordonii/8 weeks S.gordonii infection.csv',
  'inf_sg_8_weeks'),
 ('../data/Streptococcus_gordonii/8 weeks SHAM infection.csv',
  'sham_sg_8_weeks'),
 ('../data/Streptococcus_gordonii/16 weeks S. gordonii infection.csv',
  'inf_sg_16_

In [6]:
miRNA_df = make_mirna_nanostring_df(file_names, cohort_names)
miRNA_df.shape

(200, 604)

---

## create cohort dataframes

In [7]:
df_8_weeks = make_study_df(miRNA_df, cohort_str='8_weeks', infected_str='inf_')
df_16_weeks = make_study_df(miRNA_df, cohort_str='16_weeks', infected_str='inf_')
df_all_weeks = make_study_df(miRNA_df, infected_str='inf_')

### create X, y datasets

In [8]:
X_8_weeks, y_8_weeks = df_8_weeks.drop('infected', axis=1), df_8_weeks['infected']
X_16_weeks, y_16_weeks = df_16_weeks.drop('infected', axis=1), df_16_weeks['infected']
X_all_weeks, y_all_weeks = df_all_weeks.drop('infected', axis=1), df_all_weeks['infected']

---

## random forest - search for best params

In [9]:
# see https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
# n_estimators: Number of trees in random forest
# max_features: Number of features to consider at every split
# max_depth: Maximum number of levels in tree
# min_samples_split: Minimum number of samples required to split a node
# min_samples_leaf: Minimum number of samples required at each leaf node
# bootstrap: Method of selecting samples for training each tree
# oob_score: (bool) Whether to use out-of-bag samples to estimate the generalization score
param_grid = {
    'criterion' :['gini', 'entropy'],
    'n_estimators': list(range(10, 301, 10)), # [10, 20, 30, ... 300]
    'max_features': ['sqrt','log2', None],
    'max_depth': list(range(10, 201, 10)) + [None],# [10, 20, 30, ... 200, None]
    'min_samples_split': list(range(2, 11, 2)), # [2, 4, 6, ... 10]
    'min_samples_leaf': [1, 2, 4],
    'random_state': list(range(0, 9, 10)) # [0, 3, 6, 9]
}

In [10]:
# %%time
# # used for testing a single run
# print('** 8 weeks params **')
# print(f"""Best params:\n
#     {exec_HPO(RandomForestClassifier(), param_grid, X_8_weeks, y_8_weeks)}\n"""
# )

In [11]:
%%time
print('** 8 weeks params **')
print(f"""Best params:\n
    {exec_HPO(RandomForestClassifier(), param_grid, X_8_weeks, y_8_weeks)}\n"""
)

print('** 16 weeks params **')
print(f"""Best params:\n
    {exec_HPO(RandomForestClassifier(), param_grid, X_16_weeks, y_16_weeks)}\n"""
)

print('** All weeks params **')
print(f"""Best params:\n
    {exec_HPO(RandomForestClassifier(), param_grid, X_all_weeks, y_all_weeks)}\n"""
)

** 8 weeks params **
gpu-random selected
Best clf:
 RandomForestClassifier(max_depth=130, max_features='log2', min_samples_leaf=2,
                       min_samples_split=4, n_estimators=210, random_state=0) 
score (f1_score):
0.9894736842105263
---

Best params:

    {'random_state': 0, 'n_estimators': 210, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 130, 'criterion': 'gini'}

** 16 weeks params **
gpu-random selected
Best clf:
 RandomForestClassifier(max_depth=130, max_features='log2', min_samples_leaf=2,
                       min_samples_split=4, n_estimators=210, random_state=0) 
score (f1_score):
0.9789473684210526
---

Best params:

    {'random_state': 0, 'n_estimators': 210, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 130, 'criterion': 'gini'}

** All weeks params **
gpu-random selected
Best clf:
 RandomForestClassifier(criterion='entropy', max_depth=100, max_features='log2',
                     

---

## xgboost - search for best params

In [12]:
# see https://medium.com/grabngoinfo/hyperparameter-tuning-for-xgboost-91449869c57e
# see https://medium.com/@rithpansanga/the-main-parameters-in-xgboost-and-their-effects-on-model-performance-4f9833cac7c
# n_estimators: Number of trees in random forest
# max_depth: Maximum number of levels in tree
# tree_method: Tree construction algorithm used in XGBoost
# colsample_bytree: Percentage of columns to be randomly samples for each tree.
# eta: Learning rate
# gamma: Minimum loss reduction required to make further partition
# reg_alpha provides l1 regularization to the weight, higher values result in more conservative models
# reg_lambda provides l2 regularization to the weight, higher values result in more conservative models

param_grid = {
    "objective": ["binary:logistic"],
    "n_estimators": list(range(25, 1001, 25)), # [25, 50, 75, ... 1000],
    "booster": ["gbtree"], # note TreeExplainer only supports gbtree
    "learning_rate": [0.1, 0.3, 0.5, 0.7, 0.9, 1],
    "gamma": [0.0, 0.1, 0.2, 0.3, 0.4],
    'max_depth': list(range(10, 201, 10)) + [None],# [10, 20, 30, ... 200, None],
    "colsample_bytree": [i/10.0 for i in range(3,10)],
    # "reg_alpha": [1e-5, 1e-2, 0.1, 1, 10, 100], # not searching
    # "reg_lambda": [1e-5, 1e-2, 0.1, 1, 10, 100], # not seqarching
    'random_state': list(range(0, 9, 10)) # [0, 3, 6, 9]
}

In [13]:
# %%time
# # used for testing a single run
# print('** 8 weeks params **')
# print(f"""Best params:\n
#     {exec_HPO(xgb.XGBClassifier(), param_grid, X_8_weeks, y_8_weeks)}\n"""
# )

In [14]:
%%time
print('** 8 weeks params **')
print(f"""Best params:\n
    {exec_HPO(xgb.XGBClassifier(), param_grid, X_8_weeks, y_8_weeks)}\n"""
)

print('** 16 weeks params **')
print(f"""Best params:\n
    {exec_HPO(xgb.XGBClassifier(), param_grid, X_16_weeks, y_16_weeks)}\n"""
)

print('** All weeks params **')
print(f"""Best params:\n
    {exec_HPO(xgb.XGBClassifier(), param_grid, X_all_weeks, y_all_weeks)}\n"""
)

** 8 weeks params **
gpu-random selected
Best clf:
 XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.3, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.9, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=150, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=0, ...) 
score (f1_score):
0.9789473684210526
---

Best params:

    {'random_state': 0, 'objective': 'binary:logistic', 'n_estimators': 1000, 'max_depth': 150, 'learning_rate': 0.9, 'gamma': 0.3, 'col

---

## logistic regression - search for best params

In [15]:
# L2 regularization, newton-cg and lbfgs only support l2
param_grid = {
    'penalty' : ['l2'], # 'penalty' : ['l1, 'l2'] - use only l2
    'C': [0.1, 1, 10, 100, 1000], 
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
    'max_iter': [50, 100, 150, 200]
}

In [16]:
# %%time
# # used for testing a single run
# print('** 8 weeks params **')
# print(f"""Best params:\n
#     {exec_HPO(LogisticRegression(), param_grid, X_8_weeks, y_8_weeks, mode="gpu-grid")}\n"""
# )

In [17]:
%%time
print('** 8 weeks params **')
print(f"""Best params:\n
    {exec_HPO(LogisticRegression(), param_grid, X_8_weeks, y_8_weeks, mode="gpu-grid")}\n"""
)

print('** 16 weeks params **')
print(f"""Best params:\n
    {exec_HPO(LogisticRegression(), param_grid, X_16_weeks, y_16_weeks, mode="gpu-grid")}\n"""
)

print('** All weeks params **')
print(f"""Best params:\n
    {exec_HPO(LogisticRegression(), param_grid, X_all_weeks, y_all_weeks, mode="gpu-grid")}\n"""
)

** 8 weeks params **
gpu-grid selected
Best clf:
 LogisticRegression(C=0.1, max_iter=50, solver='newton-cg') 
score (f1_score):
0.9578947368421052
---

Best params:

    {'C': 0.1, 'max_iter': 50, 'penalty': 'l2', 'solver': 'newton-cg'}

** 16 weeks params **
gpu-grid selected
Best clf:
 LogisticRegression(C=1) 
score (f1_score):
0.9450292397660819
---

Best params:

    {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}

** All weeks params **
gpu-grid selected
Best clf:
 LogisticRegression(C=10, max_iter=200) 
score (f1_score):
0.9236334078439341
---

Best params:

    {'C': 10, 'max_iter': 200, 'penalty': 'l2', 'solver': 'lbfgs'}

CPU times: user 44.1 s, sys: 9.62 s, total: 53.7 s
Wall time: 32.7 s


---

## support vector machine classifer - search for best params

In [18]:
param_grid = {
    'C': [0.1,1, 10, 100, 1000], 
    'gamma': ['scale', 1,0.1,0.01,0.001, 0.0001],
    'kernel': ['rbf', 'poly', 'sigmoid']    
}

In [19]:
# %%time
# # used for testing a single run
# print('** 8 weeks params **')
# print(f"""Best params:\n
#     {exec_HPO(SVC(), param_grid, X_8_weeks, y_8_weeks, mode="gpu-grid")}\n"""
# )

In [20]:
%%time
print('** 8 weeks params **')
print(f"""Best params:\n
    {exec_HPO(SVC(), param_grid, X_8_weeks, y_8_weeks, mode="gpu-grid")}\n"""
)

print('** 16 weeks params **')
print(f"""Best params:\n
    {exec_HPO(SVC(), param_grid, X_16_weeks, y_16_weeks, mode="gpu-grid")}\n"""
)

print('** All weeks params **')
print(f"""Best params:\n
    {exec_HPO(SVC(), param_grid, X_all_weeks, y_all_weeks, mode="gpu-grid")}\n"""
)

** 8 weeks params **
gpu-grid selected
Best clf:
 SVC(C=1, gamma=1) 
score (f1_score):
1.0
---

Best params:

    {'C': 1, 'gamma': 1, 'kernel': 'rbf'}

** 16 weeks params **
gpu-grid selected
Best clf:
 SVC(C=1, gamma=1) 
score (f1_score):
1.0
---

Best params:

    {'C': 1, 'gamma': 1, 'kernel': 'rbf'}

** All weeks params **
gpu-grid selected
Best clf:
 SVC(C=1, gamma=1) 
score (f1_score):
1.0
---

Best params:

    {'C': 1, 'gamma': 1, 'kernel': 'rbf'}

CPU times: user 17.9 s, sys: 385 ms, total: 18.2 s
Wall time: 11.1 s


---

## multilayer perceptron - search for best params

In [21]:
# see https://panjeh.medium.com/scikit-learn-hyperparameter-optimization-for-mlpclassifier-4d670413042b
param_grid = {
    'hidden_layer_sizes': [(256, 128), (300, 150), (400, 200)],
    'activation': ['tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.01, 0.001, 0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive', 'invscaling']
}

In [22]:
# %%time
# # used for testing a single run
# print('** 8 weeks params **')
# print(f"""Best params:\n
#     {exec_HPO(MLPClassifier(), param_grid, X_8_weeks, y_8_weeks)}\n"""
# )

In [23]:
%%time
print('** 8 weeks params **')
print(f"""Best params:\n
    {exec_HPO(MLPClassifier(), param_grid, X_8_weeks, y_8_weeks)}\n"""
)

print('** 16 weeks params **')
print(f"""Best params:\n
    {exec_HPO(MLPClassifier(), param_grid, X_16_weeks, y_16_weeks)}\n"""
)

print('** All weeks params **')
print(f"""Best params:\n
    {exec_HPO(MLPClassifier(), param_grid, X_all_weeks, y_all_weeks)}\n"""
)

** 8 weeks params **
gpu-random selected
Best clf:
 MLPClassifier(activation='tanh', alpha=0.01, hidden_layer_sizes=(400, 200),
              solver='lbfgs') 
score (f1_score):
0.9684210526315788
---

Best params:

    {'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (400, 200), 'alpha': 0.01, 'activation': 'tanh'}

** 16 weeks params **
gpu-random selected
Best clf:
 MLPClassifier(activation='tanh', hidden_layer_sizes=(300, 150),
              learning_rate='adaptive', solver='lbfgs') 
score (f1_score):
0.9567251461988303
---

Best params:

    {'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (300, 150), 'alpha': 0.0001, 'activation': 'tanh'}

** All weeks params **
gpu-random selected
Best clf:
 MLPClassifier(activation='tanh', alpha=0.01, hidden_layer_sizes=(300, 150),
              solver='lbfgs') 
score (f1_score):
0.8999010346378767
---

Best params:

    {'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (300, 150),